In [1]:
import numpy as np
import polars as pl
from scipy.optimize import minimize

In [ ]:
# Assume your data: replace with your actual 20x20 cov matrix and 20x1 mean vector
with open("cov_m.csv") as f:
    df = pl.read_csv("cov_m.csv")

lf_sigma = [row[1:] for row in df.iter_rows()]
Sigma = np.array(lf_sigma)
shape = Sigma.shape
df_returns = pl.read_csv("Part_B_Data_530521305.csv", skip_rows=1)
tickers = df_returns.columns[1:]
r = np.array([])
mean_return_s = {}
for stock in tickers:
    mean = df_returns.select(pl.col(stock)).mean().item()
    mean_return_s[stock] = mean
    r = np.append(r, mean)

np.set_printoptions(suppress=True, precision=10)

[ 0.0329253056  0.0153364861 -0.0038536389  0.0116588611  0.0112193889
  0.0066705278  0.0075910278  0.0231497361  0.0098583056  0.0030936111
  0.0109475833  0.0160801667  0.0129607778 -0.0000780556  0.0060730972
  0.0010899861  0.0145139444  0.0147293056  0.01071725    0.0190900278]


In [ ]:
from numpy.typing import NDArray


def utility(x: NDArray, r: NDArray, Sigma: NDArray, tol: int):
    if tol == 0:  # Min variance: minimize x^T Sigma x / 2 (ignore returns)
        return 0.5 * x.T @ Sigma @ x
    else:
        risk_aversion = 0.5 * (1 / tol)
        return - (x.T @ r - risk_aversion * x.T @ Sigma @ x)  # Minimize -U to maximize U

In [ ]:
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Sum x = 1
bounds = [(0.025, None)] * 20  # x_i >= 0.025, no upper bound

# For each Tol
for tol in [0, 1, 2]:
    initial_points = [np.ones(20)/20, np.random.dirichlet(np.ones(20)) * 0.95 + 0.025/20]  # Two starting points: uniform and random (adjusted for bounds)
    
    for i, x0 in enumerate(initial_points):
        if tol == 0:
            # Min variance: minimize quadratic variance
            res = minimize(lambda x: utility(x, r, Sigma, tol=0), x0=x0,
                           method='trust-constr', bounds=bounds, constraints=constraints)
        else:
            res = minimize(lambda x: utility(x, r, Sigma, tol), x0=x0,
                           method='SLSQP', bounds=bounds, constraints=constraints)
        print(f"Tol={tol}, Start {i+1}: Optimal x = {res.x}, Utility = {-res.fun if tol > 0 else -res.fun}, Success: {res.success}")


Tol=0: Optimal x = [0.0250655174 0.0272048883 0.025046349  0.0250889572 0.0252188492
 0.0250554566 0.1190153656 0.0250438886 0.0251287188 0.0256566161
 0.1671188139 0.025072244  0.0254533846 0.0254121359 0.0803444707
 0.2284278534 0.0251878382 0.0251068221 0.0250782062 0.0252736242], Utility = -0.0007701573431978998
Tol=1: Optimal x = [0.525 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025], Utility = 0.019919753947638903
Tol=2: Optimal x = [0.525 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025], Utility = 0.020988374543263952


/home/rickhonda/uni/finc3017/portfolio-analysis-finc3017/.venv/lib/python3.13/site-packages/scipy/optimize/_differentiable_functions.py:728: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
